In [635]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [636]:
# !pip install --upgrade torchtext
# !pip install --upgrade torch

from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence
from torch import optim
import torch.nn as nn

import pandas as pd
from tqdm import tqdm

In [637]:
torch.cuda.is_available()

True

In [638]:
# from google.colab import drive
# drive.mount("/content/gdrive")


pd.set_option("display.max_colwidth", -1)
# path = "/content/gdrive/My Drive/data/amazon/amazon-reviews-max50-37k.csv"
path = "data/amazon-reviews-max50-half.csv"
df = pd.read_csv(path)

<ipython-input-638-d057213c6eb1>:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


In [639]:
df.head()

,source,target
0,"Since my first trip to Jamaica, I fell in love with the coffee. The delivery date was right on and we had Blue Mountain coffee on Christmas morning! For a treat, try drinking it with sweetened condensed milk!",Jamaican all the way!
1,"this lavish oatmeal in dark chocolate tastes great. the oatmeal is very convenient and quick to make. it tastes great, like cocoa, not exactly like a rich dark chocolate, but still yummy.",yum
2,"I was truly expecting better from Mrs. Dash, but this is nothing more than vinegary glycerin with some unrecognized flavoring. There is absolutely NO oriental flavor whatsoever. Do yourself a favor and go buy some Balsamic vinegar, it tastes the same.",Don't waste your time
3,"These are my favorite cookies, I have one almost every day. It's hard to believe that they are vegan, they taste much better than most non-vegan cookies!",I love these cookies!
4,"Two of the most yummy, sinful flavors in one healthy package, who knew? Love it for the late afternoon pick me up when in fade mode! Thank you ZICO!",decadence made healthful


In [640]:
df.describe()

,source,target
count,37089,37089
unique,32851,26076
top,"It was touted by a friend as a sleep inducer, so I tried it and it works. It tastes kinda mediciney, but it definitely does the job.",Delicious!
freq,8,207


In [641]:
len(df)

37089

In [642]:
from torchtext.data import Field, TabularDataset

TEXT = Field(use_vocab=True, init_token="<SOS>", eos_token="<EOS>",
             fix_length=25, tokenize="spacy",
             include_lengths=True, batch_first=True, lower=True,
             is_target=False
             )

fields = [("source", TEXT),
          ("target", TEXT)]

data = TabularDataset(path, "CSV", fields, skip_header=True)

/home/liefe/.virtualenvs/s2s/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/liefe/.virtualenvs/s2s/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/home/liefe/.virtualenvs/s2s/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the mos

In [643]:
train_data, test_data = data.split(.70)
train_data, val_data = train_data.split(.90)

In [644]:
len(train_data), len(val_data), len(test_data)

(23366, 2596, 11127)

Build
vocab

In [645]:
# TEXT.build_vocab(train_data, vectors="fasttext.en.300d")
# TEXT.build_vocab(train_data, vectors="fasttext.en.300d")
# TEXT.build_vocab(train_data, vectors="glove.6B.100d", max_size=10000, min_freq=3)
TEXT.build_vocab(train_data, max_size=10000, min_freq=3)

#  Next time max_size=10000, min_freq=3 ?


In [646]:
print(f"Unique tokens in source/target vocabulary: {len(TEXT.vocab)}")

Unique tokens in source/target vocabulary: 8510


In [647]:
test_data[0].__dict__.keys()

dict_keys(['source', 'target'])

In [648]:
train_data[0].source, train_data[0].target

(['save',
  'your',
  'money',
  'these',
  'wrappers',
  'were',
  'not',
  'what',
  'i',
  'expected',
  'and',
  'you',
  'ca',
  "n't",
  'return',
  'them',
  '.',
  'the',
  'taste',
  'and',
  'texture',
  'was',
  'not',
  'what',
  'i',
  'had',
  'expected',
  '.'],
 ['soy', 'wrappers'])

In [649]:
TEXT.vocab.freqs["the"], TEXT.vocab.freqs["a"], TEXT.vocab.freqs["shit"]

(29578, 20419, 0)

In [650]:
TEXT.vocab.stoi["the"], TEXT.vocab.itos[5]

(5, 'the')

In [651]:
import pprint

batch = ["Eric needs to leave".split()]
pp = pprint.PrettyPrinter(indent=4)
pad = TEXT.pad(batch)
print(pad)

([['<SOS>', 'Eric', 'needs', 'to', 'leave', '<EOS>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']], [6])


In [652]:
import random

i = random.randrange(0, len(train_data))
print(i)
source = train_data[i].source
TEXT.pad([source])

2598


([['<SOS>',
   'best',
   'flavored',
   'k',
   'cup',
   'i',
   "'ve",
   'ever',
   'had',
   '!',
   'i',
   'recommend',
   'this',
   'for',
   'everyone',
   '.',
   'great',
   'flavor',
   '!',
   'i',
   "'ve",
   'given',
   'it',
   'to',
   '<EOS>']],
 [25])

In [653]:
train_data[0].source

['save',
 'your',
 'money',
 'these',
 'wrappers',
 'were',
 'not',
 'what',
 'i',
 'expected',
 'and',
 'you',
 'ca',
 "n't",
 'return',
 'them',
 '.',
 'the',
 'taste',
 'and',
 'texture',
 'was',
 'not',
 'what',
 'i',
 'had',
 'expected',
 '.']

In [654]:
train_data[0].target

['soy', 'wrappers']

In [655]:
import random

i = random.randrange(0, len(train_data))
print(i)
source = train_data[i].source
TEXT.process([source])

13166


(tensor([[   2,    6,    0,    9, 1185,   17,  612, 1189,  307,  443,  172,    4,
             6,  142,   37,  110,   10,  132,   10,   29,   62,  398,    4,  404,
             3]]),
 tensor([25]))

Define
iterator / batcher / loader
here
note
that
this is

Model


In [656]:
class Encoder(nn.Module):
    def __init__(self, config):
        super(Encoder, self).__init__()
        self.source_dim = config["source_dim"]  # vocab size/number of classes
        self.embedding_dim = config["embedding_dim"]
        self.hidden_dim = config["hidden_dim"]
        self.layers = config.get("layers", 1)
        self.bidirectional_encoder = config.get("bidirectional_encoder", False)

        # Layers
        self.embedding = nn.Embedding(self.source_dim, self.embedding_dim)
        self.dropout = nn.Dropout(config.get("dropout", 0.0))
        self.gru = nn.GRU(self.embedding_dim, self.hidden_dim, dropout=config.get("dropout", 0.0),
                          num_layers=self.layers, bias=True, batch_first=True, bidirectional=self.bidirectional_encoder)
        # self.gru = nn.GRU(64, 64, dropout=0, num_layers=1, bias=True, batch_first=True, bidirectional=False).to(device)

    def forward(self, source, source_lens=None, pack_padded=False):

        """
        :param source: [B, T]
        :param source_lengths: [T]
        :param pack_padded: bool
        :return: output: [B, T, H], hidden: Union([1, B, H] or [2, B, H])
        """
        # print("encoder forward")
        # print(input)

        # Embed source index sequences [B, T] > [B, T, E]
        embedded = self.embedding(source)

        # print(embedded)

        # Apply dropout
        embedded = self.dropout(embedded)

        # Pack if padded sequences and lengths given

        if pack_padded:

            embedded = pack_padded_sequence(embedded, source_lens, batch_first=True, enforce_sorted=False)

        # Map embedded source [B, T, E] to Union([B, T, H] or [B, T, 2*H], Union([1, B, H] or [2, B, H])
        output, hidden = self.gru(embedded)  # TODO: add state tuple

        if pack_padded:
            output, output_lens = pad_packed_sequence(output, batch_first=True)

        # Sum outputs of bidirectional RNN:
        # [B, T, 2*H] > [B, T, H]
        if self.bidirectional_encoder:
            output = output[:, :, :self.hidden_dim] + output[:, :, self.hidden_dim:]

        return output, hidden

 

In [657]:
class Decoder(nn.Module):
    def __init__(self, config):
        super(Decoder, self).__init__()
        self.source_dim = config["source_dim"]  # vocab size/number of classes
        self.embedding_dim = config["embedding_dim"]  # Can be different from hidden!?
        self.bidirectional_encoder = config.get("bidirectional_encoder", False)
        self.hidden_dim = config["hidden_dim"]
        # self.hidden_dim = 2 * config["hidden_dim"] if self.bidirectional_encoder else config["hidden_dim"]

        # Layers
        self.embedding = nn.Embedding(self.source_dim, self.embedding_dim)
        self.dropout = nn.Dropout(config.get("dropout", 0.0))
        self.gru = nn.GRU(self.embedding_dim + self.hidden_dim, self.hidden_dim, dropout=config.get("dropout", 0.0),
                          bias=True, batch_first=True)
        self.fc = nn.Linear(self.hidden_dim, self.source_dim)  # input classes = output classes
        self.softmax = nn.Softmax(dim=1)

        # For attention
        self.source_layer = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.state_layer = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.weight_layer = nn.Linear(self.hidden_dim, 1)

    def forward(self, input, hidden, enc_outputs):
        """
        Process batch inputs at one time step and decode prediction.

        :param input: [B]
        :param hidden: [1, B, H]
        :param context_vec: [1, B, H]
        :return: prediction (B, H), hidden (1, B, H)
        """

        # print("decoder forward")
        # print(input)

        # Process the batch inputs only at this timestep t
        # Input of shape [B], but we want [B, 1]
        input = input.unsqueeze(1)

        # print(f"input of shape {input.size()}")
        # print(f"context vec of shape {context_vec.size()}")

        # Embed input [B, 1] > [B, 1, E]
        # Note embedding dimension needs to equal hidden if concatenated
        # But could map to same space
        input = self.embedding(input)
        # print(input)

        # Apply attention and get next input
        # [B, 1, 2*H]
        next_input = self.attention(input, hidden, enc_outputs)

        # Sum context vec and input ???padded = field.pad(minibatch)
        # embedded = embedded + context_vec

        # Apply dropout
        # embedded = self.dropout(embedded)

        # embedded = pack_padded_sequence(embedded, input_lens, batch_first=True, enforce_sorted=False)

        # Feed to RNN
        output, hidden = self.gru(next_input, hidden)

        # print(f"output==hidden {output[0][0]==hidden[0][0]}")

        # output, output_lens = pad_packed_sequence(output, batch_first=True)

        # Shape of output: (B, 1, H), we want (B, H)
        output = output.squeeze(1)

        # Get distribution of classes
        output = self.fc(output)

        return output, hidden

    def attention(self, input, hidden, enc_outputs):
        # Calculate attention

        # Calculate attention distribution alpha for all encoder outputs

        # # Find attention using dot product score (source X hidden_target)
        # Didn't work too well
        # # Reshape target hidden: [1, B, H] > [B, H, 1]
        # h_t = hidden.view(-1, self.hidden_dim, 1)
        # # print(f" s_t: {s_t.size()}")

        # # For batch, perform matrix mult of outputs hidden states and hidden state (s_t) at each time step
        # # [B, T, H] X [B, H, 1] = [B, T, 1]
        # # print(f" outputs: {outputs.size()}")
        # score = enc_outputs.bmm(h_t)

        # # Reshape [B, T, 1] > [B, 1, T] and take softmax
        # alphas = self.softmax(score.transpose(1,2))
        # # print(f" alphas: {alphas.size()}")

        # # Find attention. Take weighted sum of hidden states (i.e. linear combo)
        # # [B, 1, T] X [B, T, H] = [B, 1, H]
        # context_vec = alphas.bmm(enc_outputs)

        # Concatenate next input and context
        # [B, 1, H] ; [B, 1, H] = [B, 1, 2*H]
        # print(f" context: {context_vec.size()}")
        # print(f" in: {input.size()}")

        # Find attention using Bahdenah
        # tanh(Vh_s + Wh_t + b)

        # Project source layer: Vh_s
        # [B, T, H] > [B, T, H]
        source_layer = self.source_layer(enc_outputs)

        # Reshape target hidden: [1, B, H] > [B, 1, H]
        h_t = hidden.view(-1, 1, self.hidden_dim)

        # Project target hidden state to layer
        # [B, 1, H] > [B, 1, H]
        state_layer = self.state_layer(h_t)

        # tanh(Vh_s + Wh_t + b)
        # [B, T, H] > [B, T, H]
        score = torch.tanh(source_layer + state_layer)

        # Project to weight layer with 1 unit
        # [B, T, H] > [B, T, 1]
        weights = self.weight_layer(score)

        # print(weights[0])

        weights = self.softmax(weights)

        # print(weights[0])
        # print(weights.size())
        # print(torch.sum(weights, dim=1)[0])

        # Find attention. Take weighted sum of hidden states (i.e. linear combo)
        # [B, 1, T] X [B, T, H] = [B, 1, H]
        context_vec = weights.view(weights.size(0), 1, -1).bmm(enc_outputs)

        # print(context_vec.size())
        next_input = torch.cat([input, context_vec], dim=-1)

        return next_input
        # Sum hidden and context_vec
        # Didn't seem to work to well
        # hidden = hidden + context_vec.view(1, bs, -1)

        # Reshape [B, 1, H] > [1, B, H]
        # hidden = context_vec.view(1, bs, -1)

In [658]:
class Seq2Seq(nn.Module):

    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()

        self.bidirectional_encoder = config.get("bidirectional_encoder", False)
        self.softmax = nn.Softmax(dim=-1)

        # Block
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target=None, source_lens=None, pack_padded=False, teacher_force_ratio=.5, predict=False):
        """
        :param source: [B, T]
        :param target: [B, T]
        :param source_lengths: [T]
        :param pack_padded: bool
        :return: output (B, V), context (B, H), prev_hidden (B, H), weights (B, T)
        """

        target_vocab_size = self.encoder.source_dim  # here same, but may change for MT
        hidden_size = self.decoder.hidden_dim  # Pain in ass 2 * encoder hidden size

        # Encode, only need output for attention
        # out: [B, T, H], for bidirectional torch automatically sums outputs
        enc_outputs, hidden = self.encoder(source, source_lens, pack_padded)

        # Concatenate hidden states for bidirection encoder
        # [2, B, H] > [1, B, 2*H]
        # if self.bidirectional_encoder:
        #   hidden = torch.cat((hidden[0], hidden[1]), dim=-1)
        #   # Add first dim
        #   hidden = hidden.unsqueeze(0)

        if self.bidirectional_encoder:
            hidden = hidden[0] + hidden[1]
            # Add first dim
            hidden = hidden.unsqueeze(0)

        # Iterate through outputs and decode
        bs = source.size(0)

        # Train
        if not predict:


            # Get start token for batch, e.g. [SOS ...]
            target_length = target.size(1)  # includes padding (i.e max length) -- to be ignored by loss func

            # [B, 1]
            # print(f" target: {target.size()}")
            input = target[:, 0]  # SOS
            # print(f" input: {input.size()}")

            # Get all output predictions step by step and store in matrix of shape [B, T, H]
            predictions = torch.zeros(bs, target_length, target_vocab_size).to(device)
            for t in range(target_length):
                # Decode
                output, hidden = self.decoder(input, hidden, enc_outputs)

                # Get predicted output sequences for timestep
                prediction = self.softmax(output)

                # print("prediction", prediction)

                # [B, H] as softmax distribution
                predictions[:, t] = prediction
                prediction = prediction.argmax(1)

                # print(f"Predicted sequence {prediction}; Target {target[:, t]}")

                # Get next input with or without teacher forcing (correction)
                input = target[:, t] if random.random() < teacher_force_ratio else prediction

                # # Decode
                # prediction, hidden = self.decoder(input, hidden, context_vec)

                # # print("prediction", prediction)
                # predictions[:, t] = prediction
                # prediction = prediction.argmax(1)
                # # print(f"Predicted sequence {prediction}; Target {target[:, t]}")
                # input = target[:, t] if random.random() < teacher_force_ratio else prediction

        # Predict
        else:
            input = torch.ones((1, 1))  # SOS
            prediction = torch.zeros((1, 1))
            predictions = []
            while prediction.item() != 2:  # EOS
                prediction, hidden = self.decoder(input, hidden)
                predictions[:, t] = prediction
                prediction = prediction.argmax(1)
                input = prediction
                predictions.append(prediction.item())

        return predictions





In [659]:
def train(model, iterator, criterion, optimizer, epoch, loss_fun=None):
    
    model.train()
    train_loss = 0

    for i, batch in enumerate(train_iter):

        # Zero so gradients don't accumate
        optimizer.zero_grad()

        # Move tensors to cuda
        source, source_lens = batch.source
        target, target_lens = batch.target
        source = source.to(device)  # [B,T]
        target = target.to(device)  # [B,T]

        # Encode and decode sequence
        predictions = model(source, target, source_lens, pack_padded=True)

        
        assert len(predictions) == len(target)  

        #         assert len(iterator) == len(source) == len(target)
        #         n = len(source)
        
        if loss_fun is not None:
            calc_loss = loss_fun(predictions, target, target_lens)
            #               print("Calculated loss", calc_loss.item())
            #             print("Averaged loss", calc_loss.item() / n)
            loss = calc_loss




    
        # print(predictions)
        # print(target)

        # Truncate first SOS prediction and reshape [B, T, H] > [B*T, H]
        predictions = predictions[:, 1:, :].reshape(-1, predictions.size(2))
        target = target[:, 1:].reshape(-1)
        
        #         print("Torch CE loss", criterion(predictions, target).item())



        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        
        if i % 1000 == 999:
            print(f"Training loss for epoch\t {epoch}: {train_loss}")
            print(f"Averaged over 1000 epochs\t {train_loss / 1000}")
            print(f"Total steps\t {epoch * len(iterator) + i} steps")


    return train_loss

In [660]:
def eval(model, iterator, criterion, epoch, loss_fun=None):
    
    model.eval()

    with torch.no_grad():
        
        epoch_loss = 0
        for i, batch in enumerate(iterator):
            # Move tensors to cuda
            source, source_lens = batch.source
            target, target_lens = batch.target
            source = source.to(device)
            target = target.to(device)

            predictions = model(source, target, source_lens, pack_padded=True)


            if loss_fun is not None:
                calc_loss = loss_fun(predictions, target, target_lens)
                #                 print("Calculated loss", calc_loss.item())
                #             print("Averaged loss", calc_loss.item() / n)
                loss = calc_loss
                    
            # Truncate first SOS prediction
            predictions = predictions[:, 1:, :].reshape(-1, predictions.size(2)) 
            target = target[:, 1:].reshape(-1)
            
            assert len(predictions) == len(target)  # B * (T - 1)
            
            # print(f"Predictions of shape {predictions.size()}", predictions)
            # print(f"Target of shape {target.size()}", target)

            #             print("Torch ce loss", criterion(predictions, target).item())

            epoch_loss += loss.item()

            if i % 1000 == 999:
                print(f"Validation loss for epoch\t {epoch}: {train_loss}")
                print(f"Averaged over 1000 epochs\t {dev_loss / 1000}")
                print(f"Total steps\t {epoch * len(iterator) + i} steps")

        return epoch_loss


In [661]:
import random


def random_eval(model, data, r=2):
    model.eval()

    with torch.no_grad():

        for _ in range(r):

            i = random.randrange(0, len(data))
            print(f"random num: {i}")
            source, target = data[i].source, data[i].target

            print(">", " ".join(source))
            print("=", " ".join(target))

            # Perform preprocessing (indexing, etc.)
            source, source_lens = TEXT.process([source])
            target, target_lens = TEXT.process([target])
            #             print(source)
            #             print(target)
            #             print(source_lens)
            #             print(target_lens)

            # Move tensors to cuda
            source = source.to(device)
            target = target.to(device)

            predictions = model(source, target, source_lens, pack_padded=True)

            # Truncate first SOS prediction
            predictions = predictions[:, 1:, :]
            target = target[:, 1:]

            # Get predicted sequences [B, T]
            predicted_seqs = torch.argmax(predictions, -1)

            for i in range(len(predicted_seqs)):
                # for t in range(len(predicted_seqs[i])):
                print(f"  {' '.join([TEXT.vocab.itos[t] for t in predicted_seqs[i, :-1]])}")

                #             print(f"Predictions of shape {predictions.size()}", predictions)

                #             print(f"Predicted seq of shape {predicted_seqs.size()}", predicted_seqs)

                #             print(f"Target of shape {target.size()}", target)


## Retrain, set new params

In [662]:
# Instantiate model


config = dict(
    source_dim=len(TEXT.vocab),
    embedding_dim=512,
    hidden_dim=512,
    layers=1,
    dropout=.5,
    bidirectional_encoder=True,
    bs=32,
    epochs=25,
    lr=.001
)
print(f"Config: {config}")

Config: {'source_dim': 8510, 'embedding_dim': 512, 'hidden_dim': 512, 'layers': 1, 'dropout': 0.5, 'bidirectional_encoder': True, 'bs': 32, 'epochs': 25, 'lr': 0.001}


In [663]:
from torchtext.data import BucketIterator

train_iter, val_iter, test_iter = BucketIterator.splits((train_data, val_data, test_data),
                                                        batch_sizes=(
                                                            config.get("bs"), config.get("bs"), config.get("bs")),
                                                        sort_within_batch=True,
                                                        sort_key=lambda x: len(x.source),
                                                        device="cpu")

/home/liefe/.virtualenvs/s2s/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [664]:
batch = next(iter(train_iter))
batch.source[0][0]

/home/liefe/.virtualenvs/s2s/lib/python3.8/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


tensor([   2,    6,  129,   13,   38, 1248,  133, 1917,    6,  248,   10,  426,
           7,    6,   29,  739,    4,    6,   26,   10,   20,    5,  259,   41,
           3])

In [665]:

list(train_iter)[-1].source[0][0]

tensor([   2,    6,  149,  347,   27,  199,  115,   13,  172,    4,   10,   63,
           9, 1189,  944, 3572,  819,   14,   10,   34,   15,  542,    4,   83,
           3])

In [666]:

TEXT.vocab.itos[25]

'these'

In [667]:
list(train_iter)[-1].target[0][0]  # Batch > sents, lengths

tensor([   2, 2464,    5,   45,  460,   20,    5, 1455,    0,  291,    3,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1])

In [668]:
encoder = Encoder(config).to(device)
decoder = Decoder(config).to(device)
model = Seq2Seq(encoder, decoder).to(device)
# model = torch.load(path)

# for n, p in model.named_parameters():
#     if p.requires_grad:
#         print(n, p, p.size())

/home/liefe/.virtualenvs/s2s/lib/python3.8/site-packages/torch/nn/modules/rnn.py:58: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


** Training
loop **

In [670]:
criterion = nn.CrossEntropyLoss(ignore_index=1)
optimizer = optim.Adam(model.parameters())


def loss_fun(predictions, target, target_lens, reduction=torch.sum):
    """

        :param predictions: [B, T, C]
        :param target: [B, T]
        :param target_lens: [B]

        :return: loss: [B]

    """
    assert len(predictions) == len(target) == len(target_lens)
    
    # Create mask
    bs, max_len = target.size()
    mask = torch.zeros(bs, max_len)
    for i in range(bs):
        #         print(i, target_lens[i].item(), max_len)
        #         print(mask)
        mask[i, :target_lens[i].item()] = 1
    
    # Remove SOS and reshape
    predictions = predictions[:, 1:, :].reshape(-1, predictions.size(2)) # [B*(T-1), C]
    target = target[:, 1:].reshape(-1) # [B*(T-1)]
    mask = mask[:, 1:].reshape(-1)  # [B*(T-1)]
    
    # Count total sequences (excluding masked positions)
    assert len(predictions) == len(target) == len(mask)
    #     n = len(predictions)
    n = sum(mask).item()
    
    prob_correct_seqs = torch.gather(predictions, dim=1, index=target.unsqueeze(1)).cpu()
    # print(prob_correct_seqs)
    
    # Calculate NLL (do before mask to avoid inf if zero used for mask)
    prob_correct_seqs = prob_correct_seqs
    nll = -torch.log(prob_correct_seqs) 
    #     print(nll)
    
    # Reduce nll loss across batch filtered with mask
    loss = reduction(nll * mask)
    #     print(f"Loss before: {loss}; and averaged loss over {n} samples: {loss / n}")
    
    return loss / n
    

for epoch in tqdm(range(1, (config.get("epochs") + 1))):
    train_loss = train(model, train_iter, criterion, optimizer, epoch, loss_fun)
    dev_loss = eval(model, val_iter, criterion, epoch, loss_fun)
    
    print(f"\nTotal training loss for epoch {epoch}:\t {train_loss}")
    print(f"\nTotal validation loss for epoch {epoch}:\t {dev_loss}")

    print("Random prediction from training data")
    random_eval(model, train_data, 2)
    print("Random prediction from validation data")
    random_eval(model, val_data, 2)

    path = "models/model.pt"
    torch.save(model, path)


  4%|▍         | 1/25 [01:32<36:55, 92.30s/it]


Total training loss for epoch 1:	 585288.2017669678

Total validation loss for epoch 1:	 65749.14768981934
Random prediction from training data
random num: 9482
> well i purchased this item to give away at trade shows my company attend . while i was expecting a bucket of 3 different flavors . i received a bucket of the original flavor . as i chatted with a rep from amazon.com , he basically told me that your out of luck .
= no as advertised
  <unk> <EOS> <unk> <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
random num: 10482
> i highly recommend the english breakfast black tea . i 'm not real big on tea most of the time but i give this one 5 stars . do n't be shy and try it .
= celestial seasonings devonshire english breakfast black tea
  twinings tea tea tea <EOS> tea tea <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Random prediction from validation data
random n

  8%|▊         | 2/25 [03:07<35:41, 93.09s/it]


Total training loss for epoch 3:	 469744.4774246216

Total validation loss for epoch 3:	 64847.1137008667
Random prediction from training data
random num: 22666
> the buddy biscuits came in a timely fashion , and are a great product !   thank you for the great service !
= great service !
  great product <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
random num: 21559
> very healthy and very tasty -- sort of like a better nutella ( which seems much too sweet , once you have tried this ) . artisana products are clearly best of breed . amazon prices are extremely reasonable ( some of the amazon marketplace sellers were twice the price ) .
= the absolute best
  very tasty <EOS> <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Random prediction from validation data
random num: 1199
> thses cooffee pods are extra nice because there is much les

 12%|█▏        | 3/25 [04:42<34:21, 93.73s/it]

...

 92%|█████████▏| 23/25 [36:06<03:08, 94.03s/it]


Total training loss for epoch 24:	 198280.0610961914

Total validation loss for epoch 24:	 85380.07734680176
Random prediction from training data
random num: 3682
> this stuff is simular to flavored coffee mate ( the power kind ) , yet it rivals it , and some of the other private brand verions .
= simular to flavored coffee mate .
  simular to flavored mate mate . <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
random num: 4122
> i purchased this for my girlfriend who loved this stuff over the xmas period . now she has a supply which should last several months .
= best hot chocolate ever
  best hot chocolate ever <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Random prediction from validation data
random num: 797
> my husband 's favorite biscuit ( cookie ) with tea . plain but tasty . hard to find in america .
= mcvities digestive 400 g 3 pack
  a for tea <EOS> <pad

 96%|█████████▌| 24/25 [37:40<01:33, 93.94s/it]


Total training loss for epoch 25:	 199489.60291290283

Total validation loss for epoch 25:	 84598.98159790039
Random prediction from training data
random num: 1476
> we love to grind freah beans and this brand is always great . stands on its own as flavorful and strong , plus we mix it with flavors . always delivered fresh !
= fresh and strong
  fresh and strong ! <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
random num: 21939
> great baby foods !   i like the variety pack .   much cheaper than buying it from the local grocers or co - op !   i just wish they came in the squeezie pouches !
= yum !
  yum ! <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Random prediction from validation data
random num: 962
> shame on the company that bought the name and put it on this stuff made with artifical flavors , mono and di - glycerides , and egg albu

100%|██████████| 25/25 [39:14<00:00, 94.16s/it]


In [670]:
from torch.utils.tensorboard import SummaryWriter

path = 
writer = SummaryWriter("runs/"))

criterion = nn.CrossEntropyLoss(ignore_index=1)
optimizer = optim.Adam(model.parameters())


def loss_fun(predictions, target, target_lens, reduction=torch.sum):
    """

        :param predictions: [B, T, C]
        :param target: [B, T]
        :param target_lens: [B]

        :return: loss: [B]

    """
    assert len(predictions) == len(target) == len(target_lens)
    
    # Create mask
    bs, max_len = target.size()
    mask = torch.zeros(bs, max_len)
    for i in range(bs):
        #         print(i, target_lens[i].item(), max_len)
        #         print(mask)
        mask[i, :target_lens[i].item()] = 1
    
    # Remove SOS and reshape
    predictions = predictions[:, 1:, :].reshape(-1, predictions.size(2)) # [B*(T-1), C]
    target = target[:, 1:].reshape(-1) # [B*(T-1)]
    mask = mask[:, 1:].reshape(-1)  # [B*(T-1)]
    
    # Count total sequences (excluding masked positions)
    assert len(predictions) == len(target) == len(mask)
    #     n = len(predictions)
    n = sum(mask).item()
    
    prob_correct_seqs = torch.gather(predictions, dim=1, index=target.unsqueeze(1)).cpu()
    # print(prob_correct_seqs)
    
    # Calculate NLL (do before mask to avoid inf if zero used for mask)
    prob_correct_seqs = prob_correct_seqs
    nll = -torch.log(prob_correct_seqs) 
    #     print(nll)
    
    # Reduce nll loss across batch filtered with mask
    loss = reduction(nll * mask)
    #     print(f"Loss before: {loss}; and averaged loss over {n} samples: {loss / n}")
    
    return loss / n
    

for epoch in tqdm(range(1, (config.get("epochs") + 1))):
    train_loss = train(model, train_iter, criterion, optimizer, epoch, loss_fun)
    dev_loss = eval(model, val_iter, criterion, epoch, loss_fun)
    
    print(f"\nTotal training loss for epoch {epoch}:\t {train_loss}")
    print(f"\nTotal validation loss for epoch {epoch}:\t {dev_loss}")

    print("Random prediction from training data")
    random_eval(model, train_data, 2)
    print("Random prediction from validation data")
    random_eval(model, val_data, 2)

    path = "models/model-half.pt"
    torch.save(model, path)


  4%|▍         | 1/25 [01:32<36:55, 92.30s/it]


Total training loss for epoch 1:	 585288.2017669678

Total validation loss for epoch 1:	 65749.14768981934
Random prediction from training data
random num: 9482
> well i purchased this item to give away at trade shows my company attend . while i was expecting a bucket of 3 different flavors . i received a bucket of the original flavor . as i chatted with a rep from amazon.com , he basically told me that your out of luck .
= no as advertised
  <unk> <EOS> <unk> <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
random num: 10482
> i highly recommend the english breakfast black tea . i 'm not real big on tea most of the time but i give this one 5 stars . do n't be shy and try it .
= celestial seasonings devonshire english breakfast black tea
  twinings tea tea tea <EOS> tea tea <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Random prediction from validation data
random n

  8%|▊         | 2/25 [03:07<35:41, 93.09s/it]


Total training loss for epoch 3:	 469744.4774246216

Total validation loss for epoch 3:	 64847.1137008667
Random prediction from training data
random num: 22666
> the buddy biscuits came in a timely fashion , and are a great product !   thank you for the great service !
= great service !
  great product <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
random num: 21559
> very healthy and very tasty -- sort of like a better nutella ( which seems much too sweet , once you have tried this ) . artisana products are clearly best of breed . amazon prices are extremely reasonable ( some of the amazon marketplace sellers were twice the price ) .
= the absolute best
  very tasty <EOS> <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Random prediction from validation data
random num: 1199
> thses cooffee pods are extra nice because there is much les

 12%|█▏        | 3/25 [04:42<34:21, 93.73s/it]

...

 92%|█████████▏| 23/25 [36:06<03:08, 94.03s/it]


Total training loss for epoch 24:	 198280.0610961914

Total validation loss for epoch 24:	 85380.07734680176
Random prediction from training data
random num: 3682
> this stuff is simular to flavored coffee mate ( the power kind ) , yet it rivals it , and some of the other private brand verions .
= simular to flavored coffee mate .
  simular to flavored mate mate . <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
random num: 4122
> i purchased this for my girlfriend who loved this stuff over the xmas period . now she has a supply which should last several months .
= best hot chocolate ever
  best hot chocolate ever <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Random prediction from validation data
random num: 797
> my husband 's favorite biscuit ( cookie ) with tea . plain but tasty . hard to find in america .
= mcvities digestive 400 g 3 pack
  a for tea <EOS> <pad

 96%|█████████▌| 24/25 [37:40<01:33, 93.94s/it]


Total training loss for epoch 25:	 199489.60291290283

Total validation loss for epoch 25:	 84598.98159790039
Random prediction from training data
random num: 1476
> we love to grind freah beans and this brand is always great . stands on its own as flavorful and strong , plus we mix it with flavors . always delivered fresh !
= fresh and strong
  fresh and strong ! <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
random num: 21939
> great baby foods !   i like the variety pack .   much cheaper than buying it from the local grocers or co - op !   i just wish they came in the squeezie pouches !
= yum !
  yum ! <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Random prediction from validation data
random num: 962
> shame on the company that bought the name and put it on this stuff made with artifical flavors , mono and di - glycerides , and egg albu

100%|██████████| 25/25 [39:14<00:00, 94.16s/it]


In [670]:
criterion = nn.CrossEntropyLoss(ignore_index=1)
optimizer = optim.Adam(model.parameters())


def loss_fun(predictions, target, target_lens, reduction=torch.sum):
    """

        :param predictions: [B, T, C]
        :param target: [B, T]
        :param target_lens: [B]

        :return: loss: [B]

    """
    assert len(predictions) == len(target) == len(target_lens)
    
    # Create mask
    bs, max_len = target.size()
    mask = torch.zeros(bs, max_len)
    for i in range(bs):
        #         print(i, target_lens[i].item(), max_len)
        #         print(mask)
        mask[i, :target_lens[i].item()] = 1
    
    # Remove SOS and reshape
    predictions = predictions[:, 1:, :].reshape(-1, predictions.size(2)) # [B*(T-1), C]
    target = target[:, 1:].reshape(-1) # [B*(T-1)]
    mask = mask[:, 1:].reshape(-1)  # [B*(T-1)]
    
    # Count total sequences (excluding masked positions)
    assert len(predictions) == len(target) == len(mask)
    #     n = len(predictions)
    n = sum(mask).item()
    
    prob_correct_seqs = torch.gather(predictions, dim=1, index=target.unsqueeze(1)).cpu()
    # print(prob_correct_seqs)
    
    # Calculate NLL (do before mask to avoid inf if zero used for mask)
    prob_correct_seqs = prob_correct_seqs
    nll = -torch.log(prob_correct_seqs) 
    #     print(nll)
    
    # Reduce nll loss across batch filtered with mask
    loss = reduction(nll * mask)
    #     print(f"Loss before: {loss}; and averaged loss over {n} samples: {loss / n}")
    
    return loss / n
    

for epoch in tqdm(range(1, (config.get("epochs") + 1))):
    train_loss = train(model, train_iter, criterion, optimizer, epoch, loss_fun)
    dev_loss = eval(model, val_iter, criterion, epoch, loss_fun)
    
    print(f"\nTotal training loss for epoch {epoch}:\t {train_loss}")
    print(f"\nTotal validation loss for epoch {epoch}:\t {dev_loss}")

    print("Random prediction from training data")
    random_eval(model, train_data, 2)
    print("Random prediction from validation data")
    random_eval(model, val_data, 2)

    path = "models/model.pt"
    torch.save(model, path)


  4%|▍         | 1/25 [01:32<36:55, 92.30s/it]


Total training loss for epoch 1:	 585288.2017669678

Total validation loss for epoch 1:	 65749.14768981934
Random prediction from training data
random num: 9482
> well i purchased this item to give away at trade shows my company attend . while i was expecting a bucket of 3 different flavors . i received a bucket of the original flavor . as i chatted with a rep from amazon.com , he basically told me that your out of luck .
= no as advertised
  <unk> <EOS> <unk> <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
random num: 10482
> i highly recommend the english breakfast black tea . i 'm not real big on tea most of the time but i give this one 5 stars . do n't be shy and try it .
= celestial seasonings devonshire english breakfast black tea
  twinings tea tea tea <EOS> tea tea <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Random prediction from validation data
random n

  8%|▊         | 2/25 [03:07<35:41, 93.09s/it]


Total training loss for epoch 3:	 469744.4774246216

Total validation loss for epoch 3:	 64847.1137008667
Random prediction from training data
random num: 22666
> the buddy biscuits came in a timely fashion , and are a great product !   thank you for the great service !
= great service !
  great product <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
random num: 21559
> very healthy and very tasty -- sort of like a better nutella ( which seems much too sweet , once you have tried this ) . artisana products are clearly best of breed . amazon prices are extremely reasonable ( some of the amazon marketplace sellers were twice the price ) .
= the absolute best
  very tasty <EOS> <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Random prediction from validation data
random num: 1199
> thses cooffee pods are extra nice because there is much les

 12%|█▏        | 3/25 [04:42<34:21, 93.73s/it]

...

 92%|█████████▏| 23/25 [36:06<03:08, 94.03s/it]


Total training loss for epoch 24:	 198280.0610961914

Total validation loss for epoch 24:	 85380.07734680176
Random prediction from training data
random num: 3682
> this stuff is simular to flavored coffee mate ( the power kind ) , yet it rivals it , and some of the other private brand verions .
= simular to flavored coffee mate .
  simular to flavored mate mate . <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
random num: 4122
> i purchased this for my girlfriend who loved this stuff over the xmas period . now she has a supply which should last several months .
= best hot chocolate ever
  best hot chocolate ever <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Random prediction from validation data
random num: 797
> my husband 's favorite biscuit ( cookie ) with tea . plain but tasty . hard to find in america .
= mcvities digestive 400 g 3 pack
  a for tea <EOS> <pad

 96%|█████████▌| 24/25 [37:40<01:33, 93.94s/it]


Total training loss for epoch 25:	 199489.60291290283

Total validation loss for epoch 25:	 84598.98159790039
Random prediction from training data
random num: 1476
> we love to grind freah beans and this brand is always great . stands on its own as flavorful and strong , plus we mix it with flavors . always delivered fresh !
= fresh and strong
  fresh and strong ! <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
random num: 21939
> great baby foods !   i like the variety pack .   much cheaper than buying it from the local grocers or co - op !   i just wish they came in the squeezie pouches !
= yum !
  yum ! <EOS> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Random prediction from validation data
random num: 962
> shame on the company that bought the name and put it on this stuff made with artifical flavors , mono and di - glycerides , and egg albu

100%|██████████| 25/25 [39:14<00:00, 94.16s/it]


In [671]:
x=torch.tensor([1,1,0,0])
sum(x).item()

2

In [672]:
len(TEXT.vocab)

8510

In [673]:
source = torch.randint(10, (4, 10))
target = torch.randint(10, (4, 10))

In [674]:
source

tensor([[8, 7, 4, 1, 0, 9, 3, 3, 6, 2],
        [9, 3, 0, 8, 0, 3, 8, 8, 4, 6],
        [1, 0, 9, 6, 3, 7, 7, 1, 3, 4],
        [6, 5, 2, 2, 4, 9, 2, 7, 9, 7]])

In [675]:
target

tensor([[0, 2, 3, 1, 4, 5, 3, 3, 5, 9],
        [8, 6, 6, 4, 4, 1, 0, 4, 5, 1],
        [7, 4, 8, 3, 2, 5, 3, 2, 3, 9],
        [2, 7, 0, 0, 0, 7, 3, 1, 4, 1]])

In [676]:
target_lens = [4, 6, 3, 10]

In [677]:
target[:, 4:] = 1
target

tensor([[0, 2, 3, 1, 1, 1, 1, 1, 1, 1],
        [8, 6, 6, 4, 1, 1, 1, 1, 1, 1],
        [7, 4, 8, 3, 1, 1, 1, 1, 1, 1],
        [2, 7, 0, 0, 1, 1, 1, 1, 1, 1]])

In [678]:
# Create mask
bs, max_len = target.size()
mask = torch.zeros(bs, max_len)
for i in range(bs):
    mask[i, :target_lens[i]] = 1
mask

tensor([[1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [679]:
target

tensor([[0, 2, 3, 1, 1, 1, 1, 1, 1, 1],
        [8, 6, 6, 4, 1, 1, 1, 1, 1, 1],
        [7, 4, 8, 3, 1, 1, 1, 1, 1, 1],
        [2, 7, 0, 0, 1, 1, 1, 1, 1, 1]])

In [680]:
target * mask

tensor([[0., 2., 3., 1., 0., 0., 0., 0., 0., 0.],
        [8., 6., 6., 4., 1., 1., 0., 0., 0., 0.],
        [7., 4., 8., 0., 0., 0., 0., 0., 0., 0.],
        [2., 7., 0., 0., 1., 1., 1., 1., 1., 1.]])

In [681]:
# Truncate first SOS prediction and reshape [B, T, H] > [B*T, H]
predictions = predictions[:, 1:, :].view(-1, predictions.size(2))
target = target[:, 1:].view(-1)
mask = mask[:, 1:].view(-1)

NameError: name 'predictions' is not defined

In [ ]:
x = torch.rand(4, 6
               )
x

In [ ]:
ind = torch.randint(x.size(-1), (4,))
ind

In [ ]:
torch.gather(x, 1, ind.unsqueeze(-1))

In [ ]:
x = torch.zeros(10,25)
x[3,:9] = 1
x

In [ ]:
torch.log(torch.tensor(9.88e-05))


